# Actor-Critic Algorithm Report

This project is to solve CartPole problem from Gym OpenAI. Following is sample code of AC algorithm provided by https://morvanzhou.github.io/tutorials/. My understanding of each line of code is in the inline comments.

To summarize, AC algorithm uses two models, a critic which represents Q function or V function, and an actor, which updates policy in the gradient ascent direction suggested by critic. The algorithm initializes policy randomly, samples tuple of (a, r, s', a') and then update policy paramters θ ← θ + αθ * Qw(s,a) * ∇θlnπθ(a|s). Meanwhile, it computes TD error of critic, and uses TD error to update parameters of critic: w ← w + αw * δt * ∇w Qw(s,a).

While studying the sample code, I noticed incredibly strong fluctuation of reward, sometimes even dropping from nearly 300 to 100. I suspect the reason might be resulted from the update process of critic. When calculating TD error, it uses the same network to estimate both current state and next state Q values; the network can be destabilized by falling into feedback loops between target and estimated value. After long time tuning, the result is slightly better, but still not converging to a certain point. The resulting reward distribution is as following.

<img src="./reward.png" />

The following is the sample code with inline comments.

Part 1: hyper parameters

In [ ]:
# reproducible
np.random.seed(2)
tf.set_random_seed(2)  

# Hyper Parameters
OUTPUT_GRAPH = True
MAX_EPISODE = 1000
DISPLAY_REWARD_THRESHOLD = 200  # renders environment if total episode reward is greater then this threshold
MAX_EP_STEPS = 1000   # maximum time step in one episode
RENDER = False  # rendering wastes time
GAMMA = 0.9     # reward discount in TD error

# import CartPole from gym
env = gym.make('CartPole-v0')
env.seed(1)  # reproducible
env = env.unwrapped

# get number of features and number of actions from environment
N_F = env.observation_space.shape[0]
N_A = env.action_space.n

Part 2: actor network

In [ ]:
class Actor(object):
    def __init__(self, sess, n_features, n_actions, lr, global_step):
        self.sess = sess

        # placeholder for states, which is an 1-D array
        self.s = tf.placeholder(tf.float32, [1, n_features], "state")
        # placeholder for action to choose later based on probability that 
        # is generated by two fully connect layers
        self.a = tf.placeholder(tf.int32, None, "act")
        # placeholder for TD error that is generated by critic
        # this error is used to update the model of actor
        self.td_error = tf.placeholder(tf.float32, None, "td_error")  # TD_error

        # construct the two fully connected layers of actor
        with tf.variable_scope('Actor'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=20,    # number of hidden units
                activation=tf.nn.relu,
                kernel_initializer=tf.random_normal_initializer(0., .1),    # weights
                #bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            self.acts_prob = tf.layers.dense(
                inputs=l1,
                units=n_actions,    # output units
                activation=tf.nn.softmax,   # get action probabilities
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                #bias_initializer=tf.constant_initializer(0.1),  # biases
                name='acts_prob'
            )

        # construct the loss function based on the following equation:
        # θ ← θ + αθ * Aw(s,a) * ∇θlnπθ(a|s)
        with tf.variable_scope('exp_v'):
            log_prob = tf.log(self.acts_prob[0, self.a])
            self.exp_v = tf.reduce_mean(log_prob * self.td_error)  # advantage (TD_error) guided loss

        # update the network of actor
        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(-self.exp_v, global_step)  # minimize(-exp_v) = maximize(exp_v)

    def learn(self, s, a, td):
        s = s[np.newaxis, :]
        feed_dict = {self.s: s, self.a: a, self.td_error: td}
        _, exp_v = self.sess.run([self.train_op, self.exp_v], feed_dict)
        return exp_v

    # choose action based on probability generated by current state
    def choose_action(self, s):
        s = s[np.newaxis, :]
        probs = self.sess.run(self.acts_prob, {self.s: s})   # get probabilities for all actions
        return np.random.choice(np.arange(probs.shape[1]), p=probs.ravel())   # return a int

Part 3: critic network

In [ ]:
class Critic(object):
    def __init__(self, sess, n_features, lr, global_step):
        self.sess = sess

        # placeholder for states, which is an 1-D array
        self.s = tf.placeholder(tf.float32, [1, n_features], "state")
        # placeholder for value of next state
        self.v_ = tf.placeholder(tf.float32, [1, 1], "v_next")
        # placeholder for reward
        self.r = tf.placeholder(tf.float32, None, 'r')

        # construct two fully connect layers of critic
        with tf.variable_scope('Critic'):
            l1 = tf.layers.dense(
                inputs=self.s,
                units=20,  # number of hidden units
                activation=tf.nn.relu,  # None
                # have to be linear to make sure the convergence of actor.
                # But linear approximator seems hardly learns the correct Q.
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                #bias_initializer=tf.constant_initializer(0.1),  # biases
                name='l1'
            )

            self.v = tf.layers.dense(
                inputs=l1,
                units=1,  # output units
                activation=None,
                kernel_initializer=tf.random_normal_initializer(0., .1),  # weights
                #bias_initializer=tf.constant_initializer(0.1),  # biases
                name='V'
            )

        # construct TD error = reward + y*value_next_state - estimated_value_current state
        with tf.variable_scope('squared_TD_error'):
            self.td_error = self.r + GAMMA * self.v_ - self.v
            self.loss = tf.square(self.td_error)    # TD_error = (r+gamma*V_next) - V_eval
        with tf.variable_scope('train'):
            self.train_op = tf.train.AdamOptimizer(lr).minimize(self.loss)

    # function to learn critic model and returns TD error
    def learn(self, s, r, s_):
        s, s_ = s[np.newaxis, :], s_[np.newaxis, :]

        v_ = self.sess.run(self.v, {self.s: s_})
        td_error, _ = self.sess.run([self.td_error, self.train_op],
                                          {self.s: s, self.v_: v_, self.r: r})
        return td_error

Part 4: training

In [ ]:
sess = tf.Session()

global_step = tf.Variable(0, trainable=False)
learning_rate_A = tf.compat.v1.train.exponential_decay(0.0005, global_step, 300, 0.98)
learning_rate_C = tf.compat.v1.train.exponential_decay(0.001, global_step, 300, 0.98)

actor = Actor(sess, n_features=N_F, n_actions=N_A, lr=learning_rate_A, global_step=global_step)
critic = Critic(sess, n_features=N_F, lr=learning_rate_C, global_step=global_step)     # we need a good teacher, so the teacher should learn faster than the actor

sess.run(tf.global_variables_initializer())

if OUTPUT_GRAPH:
    tf.summary.FileWriter("logs/", sess.graph)

# list of running_rewards
re = []
res = []
rr = []

for i_episode in range(MAX_EPISODE):
    # get initial state
    s = env.reset()
    t = 0
    sum_r = 0
    # list of rewards in this episode
    track_r = []
    while True:
        if RENDER: env.render()
        # choose action stochastically based on current state
        a = actor.choose_action(s)
        # get next state, reward and end-flag taking the chosen action
        s_, r, done, info = env.step(a)
        # if the episode fails, reward -5
        if done: r = -1
        # append r to the list
        track_r.append(r)
        sum_r += r
        # calculate TD error based on current state, reward and next state
        td_error = critic.learn(s, r, s_)  # gradient = grad[r + gamma * V(s_) - V(s)]
        actor.learn(s, a, td_error)     # true_gradient = grad[logPi(s,a) * td_error]

        s = s_
        t += 1

        if done or t >= MAX_EP_STEPS:
            ep_rs_sum = sum(track_r)

            if 'running_reward' not in globals():
                running_reward = ep_rs_sum
            else:
                running_reward = running_reward * 0.95 + ep_rs_sum * 0.05
            if running_reward > DISPLAY_REWARD_THRESHOLD: RENDER = True  # rendering
            print("episode:", i_episode, "  reward:", int(running_reward))
            res.append(sum_r)
            rr.append(running_reward)
            break
    if i_episode % 20 == 0:
        re.append(np.mean(res[-20:]))